## Problems with SGD

__SGD의 문제점 1 - 학습속도가 느리다__

- 최저점을 향하는 과정에서 경사가 가파른 영역으로 방향을 정하기 때문에 오른쪽으로 바로 달려가는 것이 아니라 위아래로 지그재그 움직인다.


- step마다 지그재그로 왔다갔다 해서 매우 느린 속도로 학습이 진행됨.


- 고차원에선 이런 문제가 많이 발생한다.(위 그림은 2차원!). 이해가 잘 안되므로 그림을 통해 확인해보자


![](https://user-images.githubusercontent.com/36406676/53087883-8fa0a500-354b-11e9-9b60-a586952c33c8.jpg)

![](https://user-images.githubusercontent.com/36406676/53087921-a5ae6580-354b-11e9-8461-040333828f34.jpg)

&nbsp;

__SGD의 문제점2 - local minima, saddle point__

![](https://user-images.githubusercontent.com/36406676/53066154-e7221f00-3511-11e9-8d38-0bae89f7c48d.PNG)

###  local minima

local minima가 있으면 gradient가 0이 되어서 빠져나오지 못한다. dimension이 많은 경우 모든 방향에서 loss가 증가해야함. 따라서 local minima는 매우 드물게 나타납니다.

### Saddle Point

더 내려가야 하지만 gradient가 엄청 작은 값이라 느리게 진행됨. High dimension에서 더 흔히 나타나는데 한방향으로는 loss가 증가하고 한 방향으로는 loss가 내려가기 때문. saddle point 근처도 slope가 평평하여 loss가 0에 가까워 집니다. Local minima보다 자주 나타나는 문제

&nbsp;

__SGD의 문제점 3 - noisy estimate__

![](https://user-images.githubusercontent.com/36406676/53066536-aaefbe00-3513-11e9-83ec-b7dfd1546257.PNG)

Mini batch를 이용해 gradient에 대한 true information이 아닌 noisy estimate를 얻는다. 

&nbsp;

## SGD를 보완하는 여러 Optimization

## 1. SGD+Momentum

![](https://user-images.githubusercontent.com/36406676/53066847-fd7daa00-3514-11e9-8bf6-93abefe3a2b4.PNG)

SGD with Momentum은 이전의 속도와 방향성을 유지하여 다시 사용하는 것. 

In [ ]:
class SGD:
    def __init__(self, lr = 0.01):
        self.lr = lr
    
    def update(self, params, grads):
        for key in params.key():
            params[key] -= self.lr * grads[key] 

In [ ]:
class Momentum:
    def __init__(self,lr=0.01,momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
        
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.item():
                self.v[key] = np.zeros_like(val)
        # 초기값 0 
                
        for key in params.keys():
            self.v[key] = self.momentum*self.v[key] - self.lr*grads[key]      
            params[key] += self.v[key]  
            
        # self.v는 이전의 움직임을 의미.

![](https://user-images.githubusercontent.com/36406676/53086432-721e0c00-3548-11e9-8f13-c2b6bdf6b5b8.jpg)

## 2. Nesterov Momentum

![](https://user-images.githubusercontent.com/36406676/53081103-7e509c00-353d-11e9-9edf-6c44ae79433b.PNG)

- 지금 위치(붉은색 점)에서 gradiet를 계산하는 것이 아니라 Velocity로 옮긴 후 예견된 위치에서 gradient를 구함



- Convex optimization 관점에서 유용

&nbsp;

## 3. AdaGrad

- velocity 대신 gradient의 제곱(squared_term)을 사용


- 많이 바뀌는 gradietn는 큰 수로 나누고  변하지 않는 gradient는 작은 수로 나누어 속도를 빠르게 함

![](https://user-images.githubusercontent.com/36406676/53082170-a93bef80-353f-11e9-8801-44de2822b565.PNG)

![](https://user-images.githubusercontent.com/36406676/53084139-93302e00-3543-11e9-8e29-38efcd1e0ff8.jpg)

In [ ]:
class AdaGrad:
    def __init__(self, lr = 0.01):
        self.lr = lr
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.item():
                self.h[key] = np.zeros_like(val)
                
            # parameter개수만큼 0 벡터를 만들어줍니다. 이는 초기값에 활용됩니다. 
            
        for key in params.keys():
            self.h[key] += grads[key] * grads[key] 
            # 초기값은 0이지만 upfate됩니다. 
            
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key] + 1e-7)
                                                   
            # 1e -7이 위 식의 epsilon에 해당됩니다. 분모에 0에 가까운 값이 들어갈때 오류가 날 수 있으므로 이 값을 넣어줍니다.
        

![](https://user-images.githubusercontent.com/36406676/53089664-88c86100-3550-11e9-8b5b-d1bb2822a394.jpg)

## RMSProp

- squared_gradient를 축적만 하지 않고 decay rate를 도입


- decay rate = 0.9 or 0.09


- 강화학습에선 RMSProp을 많이 사용

![](https://user-images.githubusercontent.com/36406676/53088309-b14e5c00-354c-11e9-897f-657f0c2601cf.PNG)

![](https://user-images.githubusercontent.com/36406676/53089141-11de9880-354f-11e9-8cc4-55d164d76d84.jpg)

## Adam

- almost use
- velocity와 squared를 모두 사용
- first moment: momentum
- second moment : AdaGrad / RMSProp

![](https://user-images.githubusercontent.com/36406676/53090950-207b7e80-3554-11e9-9090-a68cf1533d12.PNG)

![](https://user-images.githubusercontent.com/36406676/53090837-de523d00-3553-11e9-8dff-2626e0a056da.jpg)

## Learning rate

![](https://user-images.githubusercontent.com/36406676/53091686-f0cd7600-3555-11e9-99f3-839a711aab97.PNG)

- lr 을 decay시켜 minima에 도달할 수 있도록 설정


- 처음에 no decay로 시도해보고 직접 눈으로 확인


- decay 방법은 exponential decay, 1/t decay등이 있음.

![](https://user-images.githubusercontent.com/36406676/53091781-3c801f80-3556-11e9-8ddd-7931be7aad6b.PNG)

## First-Order(1차) optimization

&nbsp;

위에서 말한 모두가 First-Order optimization

- Use gradient form linear approximation
- Step to minimize the approximation

![](https://user-images.githubusercontent.com/36406676/53091900-9680e500-3556-11e9-81f7-a820f9ef21bd.PNG)